In [131]:
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json
import folium
from folium.plugins import TimestampedGeoJson, HeatMapWithTime

pd.options.display.max_colwidth = 200

In [132]:
# file_name = 'results/greedy_50_vehicles_50_bookings_linear.csv.gz'
file_name = 'results/greedy_50_vehicles_50_bookings_linear.csv'

data = pd.read_csv(file_name, sep=';',
                   # convert column to dict
                   converters={'details': lambda v: eval(v)})

print(f'Data shape {data.shape}')

# Use only 10% of the whole dataset
# data = data[:int(data.shape[0] * 0.1)]


data.head()

Data shape (30994, 9)


,clock_time,object_type,uuid,itinerary_id,from_state,to_state,lon,lat,details
0,0,vehicle,383643c0c08d493fa2a4a52d61f097b7,NaN,offline,idling,-73.997155,40.711960,{}
1,0,vehicle,2cac348162fd4b689a1f210e253cd507,NaN,offline,idling,-73.982318,40.769531,{}
2,0,vehicle,3f7ab737810a4b5f97ac91e400a0593c,NaN,offline,idling,-73.982430,40.785004,{}
3,0,vehicle,02d409215fae4fc685f832b807b7b534,NaN,offline,idling,-74.007424,40.726010,{}
4,0,vehicle,04107820f84943849262af913deeab3e,NaN,offline,idling,-73.978013,40.758114,{}


In [133]:
details = data.details.apply(pd.Series)

details[~data.itinerary_id.isna()].head()

,dropoff,eta,pickup,dst,route_duration,route_distance,trip_duration,trip_distance,vid,stop
51,NaN,11.0,02fc4a64ed0247838ab66c562ac3fdd4,"{'lat': 40.731846, 'lon': -74.006638}",13.0,0.821,0.0,0.0,NaN,NaN
52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40a692de1f0247f0b4178a6f6b85bb86,NaN
53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40a692de1f0247f0b4178a6f6b85bb86,NaN
55,NaN,11.0,ac15d66bc6ca412a96e45b834442b175,"{'lat': 40.729687, 'lon': -74.002167}",12.0,0.811,0.0,0.0,NaN,NaN
56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,02d409215fae4fc685f832b807b7b534,NaN


In [134]:
# history of one vehicle

vehicle_uuid = data[data.object_type == 'vehicle'].uuid.values[0]

vehicle_changes = data[(data.uuid == vehicle_uuid) & (data.object_type == 'vehicle')]

vehicle_changes

,clock_time,object_type,uuid,itinerary_id,from_state,to_state,lon,lat,details
0,0,vehicle,383643c0c08d493fa2a4a52d61f097b7,NaN,offline,idling,-73.997155,40.711960,{}
354,27,vehicle,383643c0c08d493fa2a4a52d61f097b7,2cdb79afeb8044608778f541cd4c619e,idling,moving_to,-73.997171,40.712042,"{'eta': 11, 'pickup': '151f44d6a2f04476a71621b021dc4ecd', 'dst': {'lat': 40.716892, 'lon': -73.995659}, 'route_duration': 10, 'route_distance': 0.695, 'trip_duration': 0, 'trip_distance': 0.0}"
465,37,vehicle,383643c0c08d493fa2a4a52d61f097b7,2cdb79afeb8044608778f541cd4c619e,moving_to,idling,-73.995666,40.716895,"{'stop': 'arrived', 'eta': 11, 'pickup': '151f44d6a2f04476a71621b021dc4ecd', 'dst': {'lat': 40.716892, 'lon': -73.995659}, 'route_duration': 10, 'route_distance': 0.695, 'trip_duration': 10, 'trip..."
467,37,vehicle,383643c0c08d493fa2a4a52d61f097b7,2cdb79afeb8044608778f541cd4c619e,idling,moving_to,-73.995666,40.716895,"{'dropoff': '151f44d6a2f04476a71621b021dc4ecd', 'dst': {'lat': 40.731606, 'lon': -73.989235}, 'route_duration': 25, 'route_distance': 1.784, 'trip_duration': 0, 'trip_distance': 0.0}"
818,62,vehicle,383643c0c08d493fa2a4a52d61f097b7,2cdb79afeb8044608778f541cd4c619e,moving_to,idling,-73.989246,40.731591,"{'stop': 'arrived', 'dropoff': '151f44d6a2f04476a71621b021dc4ecd', 'dst': {'lat': 40.731606, 'lon': -73.989235}, 'route_duration': 25, 'route_distance': 1.784, 'trip_duration': 25, 'trip_distance'..."
...,...,...,...,...,...,...,...,...,...
30304,2121,vehicle,383643c0c08d493fa2a4a52d61f097b7,51c070c9a4cb48e38b5091e8f7c7dca9,moving_to,idling,-74.001658,40.730627,"{'stop': 'arrived', 'eta': 24, 'pickup': '555260779deb49b8a3aa0540b510ea05', 'dst': {'lat': 40.730614, 'lon': -74.001617}, 'route_duration': 26, 'route_distance': 1.854, 'trip_duration': 26, 'trip..."
30317,2121,vehicle,383643c0c08d493fa2a4a52d61f097b7,51c070c9a4cb48e38b5091e8f7c7dca9,idling,moving_to,-74.001658,40.730627,"{'dropoff': '555260779deb49b8a3aa0540b510ea05', 'dst': {'lat': 40.732887, 'lon': -73.987442}, 'route_duration': 24, 'route_distance': 1.732, 'trip_duration': 0, 'trip_distance': 0.0}"
30693,2145,vehicle,383643c0c08d493fa2a4a52d61f097b7,51c070c9a4cb48e38b5091e8f7c7dca9,moving_to,idling,-73.987473,40.732900,"{'stop': 'arrived', 'dropoff': '555260779deb49b8a3aa0540b510ea05', 'dst': {'lat': 40.732887, 'lon': -73.987442}, 'route_duration': 24, 'route_distance': 1.732, 'trip_duration': 24, 'trip_distance'..."
30723,2146,vehicle,383643c0c08d493fa2a4a52d61f097b7,eb92473344e4432690a7957852d1e0e3,idling,moving_to,-73.987473,40.732900,"{'eta': 25, 'pickup': '2b8f6fb242374159b366f3d11f364623', 'dst': {'lat': 40.741421, 'lon': -73.975548}, 'route_duration': 23, 'route_distance': 1.731, 'trip_duration': 0, 'trip_distance': 0.0}"


In [135]:
m = folium.Map(location=[40.76953, -73.98232], zoom_start=12)

now = datetime.now()
features = []

prev_pos = None
for item in vehicle_changes.itertuples():
    p1 = item.lon, item.lat
    
#     folium.Circle(
#         radius=10,
#         location=[p1[1], p1[0]],
#         color='green',
#         fill=True,
#     ).add_to(m)

    if prev_pos:
        p1 = prev_pos
        
    p2 = item.lon, item.lat

    tm = str(now + timedelta(minutes=(item.clock_time)))

#     color = 'brown'
# #     if s['details'].get('pickup'):
#     if item.details.get('pickup') is not None:
#         color = 'green'
        
        
#         folium.Circle(
#             radius=25,
#             location=[p2[-1], p2[0]],
#             color='blue',|
#             fill=True,
#         ).add_to(m)
        
#         geom = {
#             'type': 'Feature',
#             'geometry': {
#                 'type': 'LineString',
#                 'coordinates': [list(p1), list(p1)]
#             },
#             'properties': {
#                 'times': [tm, tm],
#                 'style': {
#                     'color': 'blue',
#                     'opacity': 0.6
#                 },
#                 'icon': 'circle',
#                 'iconstyle': {
#                     'radius': 25,
#                     'fillOpacity': 0.9,
#                     'iconColor': 'red'
#                 }
#             }
#         }    
#         features.append(geom)
    
    
#     if item.details.get('dropoff') is not None:
#         color = 'brown'

    geom = {
        'type': 'Feature',
        'geometry': {
            'type': 'LineString',
            'coordinates': [list(p1), list(p2)]
        },
        'properties': {
            'times': [tm, tm],
            'style': {
                'color': color,
                'opacity': 0.6
            },
            'icon': 'circle',
            'iconstyle': {
                'radius': 7,
                'fillOpacity': 0.9,
                'iconColor': 'red'
            }
        }
    }

    features.append(geom)
    
    prev_pos = p2

    folium.Circle(
        radius=100,
        location=[p1[1], p1[0]],
        color='green',
        fill=False,
    ).add_to(m)
m


In [136]:
# m = folium.Map(location=[40.76953, -73.98232])

TimestampedGeoJson(
    features,
    period='PT10M',
    add_last_point=True,
    auto_play=False, 
    loop=False, 
    max_speed=1,
    loop_button=True,
    time_slider_drag_update=True
).add_to(m)

m